In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
path = ("./resources/Team_df.csv")
com_data = pd.read_csv(path)
print(com_data.shape)
com_data.head()

(224, 23)


Team  Week Result  Rec  Home                   Opp  TmScore  \
0  Buffalo Bills     1      1  1-0     1         Chicago Bears     34.0   
1  Buffalo Bills     2      1  2-0     0    Indianapolis Colts     27.0   
2  Buffalo Bills     3      1  3-0     1  Tampa Bay Buccaneers     34.0   
3  Buffalo Bills     4      0  3-1     1     Arizona Cardinals     20.0   
4  Buffalo Bills     5      1  4-1     0      Seattle Seahawks     26.0   

   OppScore  O_1stD  O_Tot_yd  ...  D_1stD  D_Tot_Yd  D_P_Yd  D_R_Yd  D_TO  \
0      14.0    18.0     386.0  ...    24.0     322.0   188.0   134.0   2.0   
1      24.0    21.0     371.0  ...    22.0     354.0   245.0   109.0   2.0   
2      24.0    22.0     407.0  ...    27.0     446.0   411.0    35.0   1.0   
3      37.0    24.0     401.0  ...    27.0     465.0   249.0   216.0   1.0   
4      17.0    24.0     476.0  ...    22.0     354.0   262.0    92.0   2.0   

   Offence  Defense  Sp.Tms  Prediction_LR  Prediction_ADA  
0    21.58     0.85   -0.52       0.418034        0.025844  
1     7.11    -1.29   -3.09       0.418034        0.025844  
2    20.56   -15.15    3.43       0.418034        0.025844  
3     8.94   -21.72   -2.64       0.418034        0.025844  
4    13.76    -6.71   -0.05       0.418034        0.025844  

[5 rows x 23 columns]

In [3]:
# get index of every team's data
team_index = com_data['Team']

# Remove Opponent, Score, Result
super_model_data = com_data[['Team', 'Opp', 'TmScore', 'O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO', 'Home']]
#  , 'ADA_Pred_Mean', 'LOG_Prediction'

# change to season stats
season_stats = ['O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO']

super_model_data[season_stats] = super_model_data[season_stats] * 16

# standardise the data
from sklearn import preprocessing

sd_data = ['O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO']
#  , 'ADA_Pred_Mean', 'LOG_Prediction'

super_model_data[sd_data] = preprocessing.scale(super_model_data[sd_data])

#get indexs of every teams
team_index = com_data['Team']

super_model_data = pd.get_dummies(super_model_data)
super_model_data.head()

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

TmScore    O_1stD  O_Tot_yd    O_P_Yd    O_R_Yd      O_TO    D_1stD  \
0     34.0 -0.762952  0.208713  0.739647 -0.863009 -0.655336  0.856943   
1     27.0 -0.108993  0.010164  0.195539 -0.299168  0.465238  0.436910   
2     34.0  0.108993  0.486681  0.985791 -0.821243 -0.655336  1.486994   
3     20.0  0.544966  0.407262  0.325088  0.118492  0.465238  1.486994   
4     26.0  0.544966  1.400007  1.335575  0.055843 -0.655336  0.436910   

   D_Tot_Yd    D_P_Yd    D_R_Yd  ...  Opp_New Orleans Saints  \
0 -0.162220 -0.528247  0.560785  ...                       0   
1  0.236406  0.231372  0.034050  ...                       0   
2  1.382454  2.443594 -1.525087  ...                       0   
3  1.619138  0.284678  2.288477  ...                       0   
4  0.236406  0.457925 -0.324130  ...                       0   

   Opp_New York Giants  Opp_New York Jets  Opp_Philadelphia Eagles  \
0                    0                  0                        0   
1                    0                  0                        0   
2                    0                  0                        0   
3                    0                  0                        0   
4                    0                  0                        0   

   Opp_Pittsburgh Steelers  Opp_San Francisco 49ers  Opp_Seattle Seahawks  \
0                        0                        0                     0   
1                        0                        0                     0   
2                        0                        0                     0   
3                        0                        0                     0   
4                        0                        0                     1   

   Opp_Tampa Bay Buccaneers  Opp_Tennessee Titans  \
0                         0                     0   
1                         0                     0   
2                         1                     0   
3                         0                     0   
4                         0                     0   

   Opp_Washington Football Team  
0                             0  
1                             0  
2                             0  
3                             0  
4                             0  

[5 rows x 58 columns]

In [4]:
super_model_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 58 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   TmScore                       224 non-null    float64
 1   O_1stD                        224 non-null    float64
 2   O_Tot_yd                      224 non-null    float64
 3   O_P_Yd                        224 non-null    float64
 4   O_R_Yd                        224 non-null    float64
 5   O_TO                          224 non-null    float64
 6   D_1stD                        224 non-null    float64
 7   D_Tot_Yd                      224 non-null    float64
 8   D_P_Yd                        224 non-null    float64
 9   D_R_Yd                        224 non-null    float64
 10  D_TO                          224 non-null    float64
 11  Home                          224 non-null    int64  
 12  Team_Arizona Cardinals        224 non-null    uint8  
 13  Team_

In [5]:
# Data processing
X = super_model_data.drop('TmScore', axis=1).values
y = super_model_data['TmScore'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.10)

## Logistic Regressor Model

In [6]:
# Run Logistic Regression and select best mean_absolute_error 

LR_model = LogisticRegression(C=0.01, solver='liblinear')

LR_model.fit(X_train, y_train)

y_pred = LR_model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))

best_score = mean_absolute_error(y_test, y_pred)

super_model = LR_model

super_model

7.956521739130435


/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(C=0.01, solver='liblinear')

## MLP Regressor Model

In [7]:
# Run MLP Regression and select best mean_absolute_error 


MLP_model = MLPRegressor(hidden_layer_sizes=100, max_iter= 5000, random_state= 40, batch_size= 30, solver= 'lbfgs', 
                        activation= 'identity')

MLP_model.fit(X_train, y_train)

y_pred = MLP_model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))

best_score = mean_absolute_error(y_test, y_pred)
super_model = MLP_model

super_model

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


4.128767161284609


MLPRegressor(activation='identity', batch_size=30, hidden_layer_sizes=100,
             max_iter=5000, random_state=40, solver='lbfgs')

## Gradient Boosting Regressor Model

In [8]:
# Run Gradient Boosting Regression and select best mean_absolute_error 

GB_model = GradientBoostingRegressor(n_estimators= 2000, random_state= 20)

GB_model.fit(X_train, y_train)

y_pred = GB_model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))
 
if (mean_absolute_error(y_test, y_pred) < best_score):
  best_score = mean_absolute_error(y_test, y_pred)
  super_model = GB_model

super_model

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


4.979449230271352


MLPRegressor(activation='identity', batch_size=30, hidden_layer_sizes=100,
             max_iter=5000, random_state=40, solver='lbfgs')

## Random Forest Regressor Model

In [9]:
# Run Random Forest Regression and select best mean_absolute_error 

RF_model = RandomForestRegressor(n_estimators= 2000, random_state= 20)

RF_model.fit(X_train, y_train)

y_pred = RF_model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))

if (mean_absolute_error(y_test, y_pred) < best_score):
  best_score = mean_absolute_error(y_test, y_pred)
  super_model = RF_model

super_model

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


3.6469565217391304


RandomForestRegressor(n_estimators=2000, random_state=20)

In [10]:
pd.options.display.max_rows = None
"""
importance = MLP_model.feature_importances_

feature_names = super_model_data.drop('TmScore', axis=1).columns

d = {'Var_Name': feature_names, 'Imp': importance}
df = pd.DataFrame(data=d)
df = df.sort_values(by= ['Imp'], ascending = False).reset_index(drop=True)
df.head(15)
"""

"\nimportance = MLP_model.feature_importances_\n\nfeature_names = super_model_data.drop('TmScore', axis=1).columns\n\nd = {'Var_Name': feature_names, 'Imp': importance}\ndf = pd.DataFrame(data=d)\ndf = df.sort_values(by= ['Imp'], ascending = False).reset_index(drop=True)\ndf.head(15)\n"

In [11]:
# function that return the score of two teams playing

pd.options.display.max_columns = None

# Create playoff test dataset from season averages

def Praedico(home_team, away_team, super_model):
    team1 = home_team
    team2 = away_team
    
    team1_data = super_model_data[com_data['Team'] == team1].drop('TmScore', axis=1).reset_index(drop=True)
    team2_data = super_model_data[com_data['Team'] == team2].drop('TmScore', axis=1).reset_index(drop=True)
    
    week_slice = slice(9,16)
    
    #1 Remove if no team names
    team1_test = pd.DataFrame(team1_data[week_slice].mean(axis=0)).T #select week to use as average
    team1_test
    opp_columns = team1_test.filter(like='Opp').columns
    
    team1_test[opp_columns] = 0
    team1_test['Opp_' + team2] = 1
    team1_test['Home'] = 1
    
    #2
    team2_test = pd.DataFrame(team2_data[week_slice].mean(axis=0)).T #select week to use as average
    opp_columns = team2_test.filter(like='Opp').columns
    
    team2_test[opp_columns] = 0
    team2_test['Opp_' + team1] = 1
    team2_test['Home'] = 0 #remove home field advantage
    
    # head to head matchup
    team1_test[['D_1stD','D_Tot_Yd','D_P_Yd','D_R_Yd','D_TO']] = team2_test[['O_1stD','O_Tot_yd','O_P_Yd','O_R_Yd','O_TO']]
    team2_test[['D_1stD','D_Tot_Yd','D_P_Yd','D_R_Yd','D_TO']] = team1_test[['O_1stD','O_Tot_yd','O_P_Yd','O_R_Yd','O_TO']]
    
    X_Playoff_test = pd.concat([team1_test, team2_test])
    X_Playoff_test.fillna(0, inplace = True) 
    
    scores = super_model.predict(X_Playoff_test)
    print(team1, "will score", round(scores[0], 1))
    print(team2, "will score", round(scores[1], 1))
    
    if scores[0] > scores[1]:
        winner = team1
    else:
        winner = team2
        
    print(winner, "are the WINNERS!!!")
    
    return scores, winner

In [12]:
print(X_train.shape)

(201, 57)


### AFC Playoff Picture

1. Tennessee Titans (12-5), champions, AFC South. The Titans held on against the Texans in Week 18 to stay ahead of the Chiefs and clinch the No. 1 seed. They have home-field advantage in the AFC playoffs and the lone bye. The conference road to Super Bowl 56 will go through Nashviille.

2. Kansas City Chiefs (12-5), champions, AFC West. The Chiefs will settle for the No. 2 seed after beating the Broncos in Week 18 because they lost head-to-head to the Titans in Week 7. They did win Super Bowl 54 coming from this position behind the Ravens.

3. Buffalo Bills (11-6), champions, AFC East. The Bills won the division and kept this seed with a home win over the Jets. They finish a game behind the co-AFC leaders and a game ahead of the Bengals after being the AFC's No. 2 in last year's playoffs.

4. Cincinnati Bengals (10-7), champions, AFC North. The Bengals rested key players because of injuries and other reasons and lost without Joe Burrow and Joe Mixon at the Browns in Week 18. They were set to stay here behind the East champions.

5. Las Vegas Raiders (10-7), second place, AFC West. The Raiders moved into playoff position with the Colts, whom they beat in Week 17, being upset by the Jaguars in Week 18. They earned a playoff berth with the wild overtime win over the Chargers on Sunday night.

6. New England Patriots (10-7), second place, AFC East. The Patriots lost to the Dolphins in Week 18, but they had already lost the East title when the Bills beat the Jets. They dropped to No. 6 with the Raiders beating the Chargers on Sunday night because of losing the tiebreaker.

7. Pittsburgh Steelers (9-7-1), second place, AFC North. The Steelers got into the playoffs to extend the career of Ben Roethlisberger by beating the Ravens in Week 18 while the Jaguars beat the Colts and the Raiders beat the Chargers.



In [13]:
# Creat a list of AFC teams

afc_playoff_picture = ['Tennessee Titans', 'Kansas City Chiefs', 'Buffalo Bills', 'Cincinnati Bengals', 'Las Vegas Raiders', 'New England Patriots', 'Pittsburgh Steelers']

### NFC Playoff Picture

1. Green Bay Packers (13-4), champions, NFC North. The Packers clinched the No. 1 seed, the lone bye and the Lambeau home-field advantage in the NFC playoffs with their win over the Vikings in Week 17, matching their win total of the previous two seasons with Aaron Rodgers under Matt LaFleur with one more game left in the new schedule. They lost in Week 18 while resting Rodgers and other regulars in the second half against the Lions.

2. Tampa Bay Buccaneers (13-4), champions, NFC South. The Buccaneers beat the Panthers and moved up to No. 2 because the Rams, to whom they lost in Week 3, lost to the 49ers. They finished behind the Packers because of a lesser confference record and ahead of the Cowboys, up a full game and a head-to-head tiebreaker from Week 1.

3. Dallas Cowboys (12-5), champions, NFC East. The Cowboys rebounded to rout the resting Eagles in Week 18 after they lost to the Cardinals in a failed comeback in Week 17. They were able to jump the losing Rams because of winning the conference-record tiebreaker over them.

4. Los Angeles Rams (12-5), champions, NFC West. The Rams failed to beat the 49ers in Week 18 but still took back the division crown with the Cardinals losing another West matchup to the Seahawks at home. They cost themselves a No. 2 seed and now need to play a third game against the Cardinals.

5. Arizona Cardinals (11-6), second place, NFC West. The Cardinals could have won the division with the Rams losing but they also lost to the Seahawks. They will settle for the top wild-card spot and a rematch with the Rams in the wild-card playoffs.

6. San Francisco 49ers (10-7), third place, NFC West. The 49ers locked down the second wild card by beating the Rams in overtime in Week 18. They held off the winning Saints and got up a full game on the losing Eagles, whom they also beat in Week 2.

7. Philadelphia Eagles (9-8), second place, NFC East. The Eagles rested players against the Cowboys in Week 18 after they had already clinched a wild-card spot in Week 17. They stayed ahead of the winning Saints because of beating that team in Week 11.

In [14]:
# Create a list of NFC teams
nfc_playoff_picture = ['Green Bay Packers', 'Tampa Bay Buccaneers', 'Dallas Cowboys', 'Los Angeles Rams', 'Arizona Cardinals', 'San Francisco 49ers', 'Philadelphia Eagles'] 

In [15]:
# Remove top seed from list
afc_divisional = []
afc_divisional.append(afc_playoff_picture.pop(0))

In [16]:
# Remove top seet from list

nfc_divisional = []
nfc_divisional.append(nfc_playoff_picture.pop(0))

### AFC Playoff Wildcard Round 

In [17]:
# Create a loop

while len(afc_playoff_picture) > 0:
    first_element = afc_playoff_picture.pop(0)
    last_element = afc_playoff_picture.pop(-1)
    score, team = Praedico(first_element, last_element, super_model)
    afc_divisional.append(team)

Kansas City Chiefs will score 26.5
Pittsburgh Steelers will score 16.6
Kansas City Chiefs are the WINNERS!!!
Buffalo Bills will score 20.1
New England Patriots will score 30.9
New England Patriots are the WINNERS!!!
Cincinnati Bengals will score 24.6
Las Vegas Raiders will score 26.4
Las Vegas Raiders are the WINNERS!!!


### NFC Playoff Wildcard Round 

In [18]:
# Create a loop

while len(nfc_playoff_picture) > 0:
    first_element = nfc_playoff_picture.pop(0)
    last_element = nfc_playoff_picture.pop(-1)
    score, team = Praedico(first_element, last_element, super_model)
    nfc_divisional.append(team)

Tampa Bay Buccaneers will score 28.9
Philadelphia Eagles will score 25.2
Tampa Bay Buccaneers are the WINNERS!!!
Dallas Cowboys will score 26.9
San Francisco 49ers will score 28.4
San Francisco 49ers are the WINNERS!!!
Los Angeles Rams will score 21.5
Arizona Cardinals will score 24.9
Arizona Cardinals are the WINNERS!!!


In [19]:
# Print the AFC Divisional list created 
afc_divisional

['Tennessee Titans',
 'Kansas City Chiefs',
 'New England Patriots',
 'Las Vegas Raiders']

In [20]:
# Print the NFC Divisional list created 
nfc_divisional

['Green Bay Packers',
 'Tampa Bay Buccaneers',
 'San Francisco 49ers',
 'Arizona Cardinals']

In [21]:
# Set AFC Champion to a empty list
afc_championship = []

In [22]:
# Set NFC Champion to a empty list
nfc_championship = []

### AFC Divisional Round 

In [23]:
while len(afc_divisional) > 0:
    first_element = afc_divisional.pop(0)
    last_element = afc_divisional.pop(-1)
    score, team = Praedico(first_element, last_element, super_model)
    afc_championship.append(team)

Tennessee Titans will score 25.4
Las Vegas Raiders will score 25.0
Tennessee Titans are the WINNERS!!!
Kansas City Chiefs will score 23.0
New England Patriots will score 28.1
New England Patriots are the WINNERS!!!


### NFC Divisional Round

In [24]:
while len(nfc_divisional) > 0:
    first_element = nfc_divisional.pop(0)
    last_element = nfc_divisional.pop(-1)
    score, team = Praedico(first_element, last_element, super_model)
    nfc_championship.append(team)

Green Bay Packers will score 24.5
Arizona Cardinals will score 22.9
Green Bay Packers are the WINNERS!!!
Tampa Bay Buccaneers will score 28.5
San Francisco 49ers will score 26.7
Tampa Bay Buccaneers are the WINNERS!!!


### AFC Championship

In [25]:
score, afc_team = Praedico(afc_championship.pop(0), afc_championship.pop(-1), super_model)

Tennessee Titans will score 15.8
New England Patriots will score 35.2
New England Patriots are the WINNERS!!!


### NFC Championship

In [26]:
score, nfc_team = Praedico(nfc_championship.pop(0), nfc_championship.pop(-1), super_model)

Green Bay Packers will score 26.7
Tampa Bay Buccaneers will score 25.9
Green Bay Packers are the WINNERS!!!


### Super Bowl LVI

In [27]:
score, super_bowl_team = Praedico(nfc_team, afc_team, super_model)
print(f'Congratulations to the {super_bowl_team} for winning Super Bowl LVI!')

Green Bay Packers will score 24.4
New England Patriots will score 28.1
New England Patriots are the WINNERS!!!
Congratulations to the New England Patriots for winning Super Bowl LVI!


In [28]:
print(f'The super_model with the lowest mean absolute error used was {super_model}.')

The super_model with the lowest mean absolute error used was RandomForestRegressor(n_estimators=2000, random_state=20).
